### Install needed dependencies
- `xpress` to import the CPLEX solver

*Note: first we need to install `numpy` being a needed dependency of the previous libraries*


In [1]:
!pip install numpy

In [2]:
!pip install xpress

### Example problem to test the xpress library was correctly installed

In [3]:
import xpress as xp
p = xp.problem(name='example')  # problem name (optional)
x1 = p.addVariable(vartype=xp.integer, name='x1', lb=-10, ub=10)
x2 = p.addVariable(name='x2')
p.setObjective(x1**2 + 2*x2)      # objective function
p.addConstraint(x1 + 3*x2 >= 4)   # one or more constraints
p.optimize()
print ("solution: {0} = {1}; {2} = {3}".format (x1.name, p.getSolution(x1), x2.name, p.getSolution(x2)))

FICO Xpress v9.5.0, Community, solve started 21:28:07, Dec 13, 2024
Heap usage: 387KB (peak 387KB, 102KB system)
Minimizing MIQP example using up to 8 threads and up to 8087MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
         1 rows            2 cols            2 elements         1 entities
         1 qobjelem
Presolved problem has:
         1 rows            2 cols            2 elements         1 entities
         1 qobjelem
Presolve finished in 0 seconds
Heap usage: 416KB (peak 431KB, 102KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  3.00e+00] / [ 1.00e+00,  3.00e+00]
  RHS and bounds [min,max] : [ 4.00e+00,  1.00e+01] / [ 4.00e+00,  1.00e+01]
  Objective      [min,max] : [ 2.00e+00,  2.00e+00] / [ 2.00e+00,  2.00e+00]
  Quadratic      [min,max] : [ 2.00e+00,  2.00e+00] / [ 2.00e+00,  2.00e+00]
Autoscaling appl

C:\Users\P2001\AppData\Local\Temp\ipykernel_3488\3312701003.py:2: LicenseWarning: Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('c:/Users/P2001/anaconda3/envs/mip/lib/site-packages/xpress/license/community-xpauth.xpr')
  p = xp.problem(name='example')  # problem name (optional)


### Example of loading and mps problem

In [1]:
import xpress as xp

# Path to the specific .mps file
file_path = "input/example.mps"

# Create a new Xpress model
model = xp.problem()

# Load the .mps file
try:
    model.read(file_path)
    print(f"Successfully loaded {file_path}")
except Exception as e:
    print(f"Error loading file {file_path}: {e}")
    exit()

Reading Problem 22433
Problem Statistics
         198 (      1 spare) rows
         429 (      0 spare) structural columns
        3408 (      1 spare) non-zero elements
MIP Entity Statistics
         231 entities        0 sets        0 set members
Successfully loaded input/example.mps


C:\Users\P2001\AppData\Local\Temp\ipykernel_10932\1080014326.py:7: LicenseWarning: Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('c:/Users/P2001/anaconda3/envs/mip/lib/site-packages/xpress/license/community-xpauth.xpr')
  model = xp.problem()


In [3]:
import xpress as xp

# Path to the specific .mps file
file_path = "input/example.mps"

# Create a new Xpress model
model = xp.problem()

# Load the .mps file
try:
    model.read(file_path)  # Load the file
    print(f"Successfully loaded {file_path}")
except Exception as e:
    print(f"Error loading file {file_path}: {e}")
    exit()

# Solve the problem
try:
    model.solve()
    print(f"Solving completed for {file_path}")
except Exception as e:
    print(f"Error solving file {file_path}: {e}")
    exit()

# Check and display results
status = model.getProbStatus()

# Interpret solver status based on Xpress documentation
if status == 0:  # Status 0 typically means 'optimal'
    print(f"Optimal solution found for {file_path}!")
    print(f"Objective Value: {model.getObjVal()}")
    solution = model.getSolution()
    for idx, value in enumerate(solution):
        print(f"Variable {idx}: {value}")
elif status == 1:  # Status 1 might indicate 'infeasible'
    print(f"The problem is infeasible for {file_path}.")
elif status == 2:  # Status 2 might indicate 'unbounded'
    print(f"The problem is unbounded for {file_path}.")
else:
    print(f"Solver finished with status {status}.")


Reading Problem 22433
Problem Statistics
         198 (      1 spare) rows
         429 (      0 spare) structural columns
        3408 (      1 spare) non-zero elements
MIP Entity Statistics
         231 entities        0 sets        0 set members
Successfully loaded input/example.mps
FICO Xpress v9.5.0, Community, solve started 21:31:18, Dec 13, 2024
Heap usage: 491KB (peak 491KB, 94KB system)
Minimizing MILP 22433 using up to 8 threads and up to 8087MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
       198 rows          429 cols         3408 elements       231 entities
Presolved problem has:
       198 rows          231 cols         3210 elements       231 entities
Presolve finished in 0 seconds
Heap usage: 829KB (peak 957KB, 94KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.05e+03] / [ 3.91e-02,  1.03e+00]
  RH

C:\Users\P2001\AppData\Local\Temp\ipykernel_10932\2514539913.py:26: DeprecationWarning: Deprecated in Xpress 9.5: use problem.attributes.solvestatus and problem.attributes.solstatus instead
  status = model.getProbStatus()
